Below: the stuff from huggingface deep rl lesson 7

In [4]:
import gym
import pybullet_envs

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

import torch 
from torch import nn

In [5]:
env_id = "AntBulletEnv-v0"
# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space

pybullet build time: May 20 2022 19:45:31


In [6]:
env = make_vec_env(env_id, n_envs=4)

# Adding this wrapper to normalize the observation and the reward
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)

In [7]:
model = A2C(policy = "MlpPolicy",
            env = env,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 0.00096,
            max_grad_norm = 0.5,
            n_steps = 8,
            vf_coef = 0.4,
            ent_coef = 0.0,
            tensorboard_log = "./tensorboard",
            policy_kwargs=dict(
            log_std_init=-2, ortho_init=False),
            normalize_advantage=False,
            use_rms_prop= True,
            use_sde= True,
            verbose=1)

Using cuda device


In [8]:
#Takes tens of minutes
model.learn(2_000_000)
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-AntBulletEnv-v0")
env.save("vec_normalize.pkl")

Logging to ./tensorboard/A2C_1
------------------------------------
| time/                 |          |
|    fps                | 1315     |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 3200     |
| train/                |          |
|    entropy_loss       | -2.05    |
|    explained_variance | 0.781    |
|    learning_rate      | 0.00096  |
|    n_updates          | 99       |
|    policy_loss        | 1.38     |
|    std                | 0.134    |
|    value_loss         | 2.02     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 480      |
|    ep_rew_mean        | 243      |
| time/                 |          |
|    fps                | 1478     |
|    iterations         | 200      |
|    time_elapsed       | 4        |
|    total_timesteps    | 6400     |
| train/                |          |
|    entropy_loss       | -2.49    |
|    ex

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 359      |
|    ep_rew_mean        | 169      |
| time/                 |          |
|    fps                | 1667     |
|    iterations         | 1400     |
|    time_elapsed       | 26       |
|    total_timesteps    | 44800    |
| train/                |          |
|    entropy_loss       | -3.35    |
|    explained_variance | 0.107    |
|    learning_rate      | 0.00096  |
|    n_updates          | 1399     |
|    policy_loss        | -1.84    |
|    std                | 0.127    |
|    value_loss         | 0.472    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 376      |
|    ep_rew_mean        | 180      |
| time/                 |          |
|    fps                | 1669     |
|    iterations         | 1500     |
|    time_elapsed       | 28       |
|    total_timesteps    | 48000    |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 627      |
|    ep_rew_mean        | 325      |
| time/                 |          |
|    fps                | 1688     |
|    iterations         | 2700     |
|    time_elapsed       | 51       |
|    total_timesteps    | 86400    |
| train/                |          |
|    entropy_loss       | -2.53    |
|    explained_variance | 0.408    |
|    learning_rate      | 0.00096  |
|    n_updates          | 2699     |
|    policy_loss        | -3.69    |
|    std                | 0.12     |
|    value_loss         | 7.95     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 642      |
|    ep_rew_mean        | 340      |
| time/                 |          |
|    fps                | 1690     |
|    iterations         | 2800     |
|    time_elapsed       | 53       |
|    total_timesteps    | 89600    |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 790      |
|    ep_rew_mean        | 454      |
| time/                 |          |
|    fps                | 1697     |
|    iterations         | 4000     |
|    time_elapsed       | 75       |
|    total_timesteps    | 128000   |
| train/                |          |
|    entropy_loss       | 0.946    |
|    explained_variance | 0.823    |
|    learning_rate      | 0.00096  |
|    n_updates          | 3999     |
|    policy_loss        | -0.134   |
|    std                | 0.114    |
|    value_loss         | 0.251    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 808      |
|    ep_rew_mean        | 468      |
| time/                 |          |
|    fps                | 1696     |
|    iterations         | 4100     |
|    time_elapsed       | 77       |
|    total_timesteps    | 131200   |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 832      |
|    ep_rew_mean        | 539      |
| time/                 |          |
|    fps                | 1694     |
|    iterations         | 5300     |
|    time_elapsed       | 100      |
|    total_timesteps    | 169600   |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | 0.978    |
|    learning_rate      | 0.00096  |
|    n_updates          | 5299     |
|    policy_loss        | 1.5      |
|    std                | 0.109    |
|    value_loss         | 1.22     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 861      |
|    ep_rew_mean        | 558      |
| time/                 |          |
|    fps                | 1695     |
|    iterations         | 5400     |
|    time_elapsed       | 101      |
|    total_timesteps    | 172800   |
|

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 1.53e+03 |
| time/                 |          |
|    fps                | 1734     |
|    iterations         | 57300    |
|    time_elapsed       | 1056     |
|    total_timesteps    | 1833600  |
| train/                |          |
|    entropy_loss       | -3.34    |
|    explained_variance | 1        |
|    learning_rate      | 0.00096  |
|    n_updates          | 57299    |
|    policy_loss        | 1.53     |
|    std                | 0.0547   |
|    value_loss         | 2.46     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 1.53e+03 |
| time/                 |          |
|    fps                | 1734     |
|    iterations         | 57400    |
|    time_elapsed       | 1058     |
|    total_timesteps    | 1836800  |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 1.4e+03  |
| time/                 |          |
|    fps                | 1735     |
|    iterations         | 58600    |
|    time_elapsed       | 1080     |
|    total_timesteps    | 1875200  |
| train/                |          |
|    entropy_loss       | -3.28    |
|    explained_variance | 0.748    |
|    learning_rate      | 0.00096  |
|    n_updates          | 58599    |
|    policy_loss        | 5.66     |
|    std                | 0.0554   |
|    value_loss         | 5.7      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 1.39e+03 |
| time/                 |          |
|    fps                | 1735     |
|    iterations         | 58700    |
|    time_elapsed       | 1082     |
|    total_timesteps    | 1878400  |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 1.34e+03 |
| time/                 |          |
|    fps                | 1736     |
|    iterations         | 59900    |
|    time_elapsed       | 1103     |
|    total_timesteps    | 1916800  |
| train/                |          |
|    entropy_loss       | -2.79    |
|    explained_variance | 0.956    |
|    learning_rate      | 0.00096  |
|    n_updates          | 59899    |
|    policy_loss        | 18.2     |
|    std                | 0.0557   |
|    value_loss         | 60.4     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 993      |
|    ep_rew_mean        | 1.31e+03 |
| time/                 |          |
|    fps                | 1736     |
|    iterations         | 60000    |
|    time_elapsed       | 1105     |
|    total_timesteps    | 1920000  |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 993      |
|    ep_rew_mean        | 1.31e+03 |
| time/                 |          |
|    fps                | 1737     |
|    iterations         | 61200    |
|    time_elapsed       | 1127     |
|    total_timesteps    | 1958400  |
| train/                |          |
|    entropy_loss       | -3.42    |
|    explained_variance | 0.997    |
|    learning_rate      | 0.00096  |
|    n_updates          | 61199    |
|    policy_loss        | 2.75     |
|    std                | 0.0555   |
|    value_loss         | 6.41     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 993      |
|    ep_rew_mean        | 1.31e+03 |
| time/                 |          |
|    fps                | 1737     |
|    iterations         | 61300    |
|    time_elapsed       | 1128     |
|    total_timesteps    | 1961600  |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 993      |
|    ep_rew_mean        | 1.2e+03  |
| time/                 |          |
|    fps                | 1738     |
|    iterations         | 62500    |
|    time_elapsed       | 1150     |
|    total_timesteps    | 2000000  |
| train/                |          |
|    entropy_loss       | -2.34    |
|    explained_variance | 0.912    |
|    learning_rate      | 0.00096  |
|    n_updates          | 62499    |
|    policy_loss        | -1.98    |
|    std                | 0.0555   |
|    value_loss         | 3.11     |
------------------------------------


End Huggingface stuff.

Now let's do it with cartpole to see how quickly it grasps a simple env.

Was going to adapt the huggingface stuff but this is from the stable baselines website verbatim

In [6]:
import gym

#from stable_baselines3.common.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C

# Parallel environments
env = make_vec_env('CartPole-v1', n_envs=4)

model = A2C("MlpPolicy", env, verbose=1)
for i in range(1000):
    model.learn(total_timesteps=1000)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    if mean_reward > 110:
        print(f"took {i * 1000} timesteps to reach mean reward 100")
        break;
#model.learn(total_timesteps=100000)
model.save("a2c_cartpole")

del model # remove to demonstrate saving and loading

model = A2C.load("a2c_cartpole")

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

Using cuda device
took 2000 timesteps to reach mean reward 100


KeyboardInterrupt: 

OK now let's try Pong

In [13]:
import gym

#from stable_baselines3.common.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C

# Parallel environments
env = make_vec_env('ALE/Pong-v5', n_envs=4)

model = A2C("CnnPolicy", env, verbose=1)
#model = A2C.load("a2c_pong")

from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%Y%m%d%H%M")
save_file = f"a2c_pong-{dt_string}"
print(f"Save file: {save_file}")

reward_milestones = []
reward_target = -20

Using cuda device
Wrapping the env in a VecTransposeImage.
Save file: a2c_pong-202208131344


In [37]:
n = 100_000
for i in range(1000):
    model.learn(total_timesteps=n, reset_num_timesteps=False)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    print(f"Now at {n * (i+1)} timesteps")
    if mean_reward > reward_target:
        print(f"took {n * (i+1)} timesteps to reach mean reward {reward_target}")
        reward_milestones.append({"reward": reward_target, "timesteps": (n*(i+1))})
        reward_target+=1
        break;

#model.learn(total_timesteps=1_000_000)

model.save(save_file)


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.48e+03 |
|    ep_rew_mean        | -4.45    |
| time/                 |          |
|    fps                | 1312     |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 10502000 |
| train/                |          |
|    entropy_loss       | -0.0601  |
|    explained_variance | 0.786    |
|    learning_rate      | 0.0007   |
|    n_updates          | 525099   |
|    policy_loss        | 0.00541  |
|    value_loss         | 0.16     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.48e+03 |
|    ep_rew_mean        | -4.45    |
| time/                 |          |
|    fps                | 1305     |
|    iterations         | 200      |
|    time_elapsed       | 3        |
|    total_timesteps    | 10504000 |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.47e+03 |
|    ep_rew_mean        | -4.31    |
| time/                 |          |
|    fps                | 1337     |
|    iterations         | 1400     |
|    time_elapsed       | 20       |
|    total_timesteps    | 10528000 |
| train/                |          |
|    entropy_loss       | -0.0442  |
|    explained_variance | 0.989    |
|    learning_rate      | 0.0007   |
|    n_updates          | 526399   |
|    policy_loss        | 0.00334  |
|    value_loss         | 0.0146   |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.46e+03  |
|    ep_rew_mean        | -4.61     |
| time/                 |           |
|    fps                | 1337      |
|    iterations         | 1500      |
|    time_elapsed       | 22        |
|    total_timesteps    | 10530000  |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.11e+03 |
|    ep_rew_mean        | -13.6    |
| time/                 |          |
|    fps                | 1339     |
|    iterations         | 2700     |
|    time_elapsed       | 40       |
|    total_timesteps    | 10554000 |
| train/                |          |
|    entropy_loss       | -0.572   |
|    explained_variance | -0.131   |
|    learning_rate      | 0.0007   |
|    n_updates          | 527699   |
|    policy_loss        | -0.158   |
|    value_loss         | 0.0724   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.11e+03 |
|    ep_rew_mean        | -13.6    |
| time/                 |          |
|    fps                | 1340     |
|    iterations         | 2800     |
|    time_elapsed       | 41       |
|    total_timesteps    | 10556000 |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.09e+03 |
|    ep_rew_mean        | -14.2    |
| time/                 |          |
|    fps                | 1339     |
|    iterations         | 4100     |
|    time_elapsed       | 61       |
|    total_timesteps    | 10582000 |
| train/                |          |
|    entropy_loss       | -0.341   |
|    explained_variance | 0.774    |
|    learning_rate      | 0.0007   |
|    n_updates          | 529099   |
|    policy_loss        | -0.0316  |
|    value_loss         | 0.0137   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.1e+03  |
|    ep_rew_mean        | -14.2    |
| time/                 |          |
|    fps                | 1335     |
|    iterations         | 4200     |
|    time_elapsed       | 62       |
|    total_timesteps    | 10584000 |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.1e+03  |
|    ep_rew_mean        | -14.1    |
| time/                 |          |
|    fps                | 1294     |
|    iterations         | 400      |
|    time_elapsed       | 6        |
|    total_timesteps    | 10608000 |
| train/                |          |
|    entropy_loss       | -0.357   |
|    explained_variance | -3.65    |
|    learning_rate      | 0.0007   |
|    n_updates          | 530399   |
|    policy_loss        | 0.0106   |
|    value_loss         | 0.0635   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.1e+03  |
|    ep_rew_mean        | -14.1    |
| time/                 |          |
|    fps                | 1302     |
|    iterations         | 500      |
|    time_elapsed       | 7        |
|    total_timesteps    | 10610000 |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 959       |
|    ep_rew_mean        | -19.6     |
| time/                 |           |
|    fps                | 1341      |
|    iterations         | 1700      |
|    time_elapsed       | 25        |
|    total_timesteps    | 10634000  |
| train/                |           |
|    entropy_loss       | -4.89e-05 |
|    explained_variance | 0.992     |
|    learning_rate      | 0.0007    |
|    n_updates          | 531699    |
|    policy_loss        | -2.6e-08  |
|    value_loss         | 0.00246   |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 959       |
|    ep_rew_mean        | -19.6     |
| time/                 |           |
|    fps                | 1342      |
|    iterations         | 1800      |
|    time_elapsed       | 26        |
|    total_timesteps    | 10636000  |
| train/    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | -18.9    |
| time/                 |          |
|    fps                | 1342     |
|    iterations         | 3000     |
|    time_elapsed       | 44       |
|    total_timesteps    | 10660000 |
| train/                |          |
|    entropy_loss       | -0.101   |
|    explained_variance | 0.663    |
|    learning_rate      | 0.0007   |
|    n_updates          | 532999   |
|    policy_loss        | 0.000518 |
|    value_loss         | 0.0212   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.01e+03 |
|    ep_rew_mean        | -18.9    |
| time/                 |          |
|    fps                | 1343     |
|    iterations         | 3100     |
|    time_elapsed       | 46       |
|    total_timesteps    | 10662000 |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.07e+03 |
|    ep_rew_mean        | -16.9    |
| time/                 |          |
|    fps                | 1335     |
|    iterations         | 4300     |
|    time_elapsed       | 64       |
|    total_timesteps    | 10686000 |
| train/                |          |
|    entropy_loss       | -0.434   |
|    explained_variance | 0.689    |
|    learning_rate      | 0.0007   |
|    n_updates          | 534299   |
|    policy_loss        | -0.0747  |
|    value_loss         | 0.0362   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.07e+03 |
|    ep_rew_mean        | -16.9    |
| time/                 |          |
|    fps                | 1335     |
|    iterations         | 4400     |
|    time_elapsed       | 65       |
|    total_timesteps    | 10688000 |
| train/                |          |
|

In [38]:
print(reward_milestones)
print(f"Next reward target: {reward_target}")

[{'reward': -20, 'timesteps': 900000}, {'reward': -19, 'timesteps': 100000}, {'reward': -18, 'timesteps': 100000}, {'reward': -17, 'timesteps': 200000}, {'reward': -16, 'timesteps': 100000}, {'reward': -15, 'timesteps': 100000}, {'reward': -14, 'timesteps': 200000}, {'reward': -13, 'timesteps': 200000}, {'reward': -12, 'timesteps': 300000}, {'reward': -11, 'timesteps': 100000}, {'reward': -10, 'timesteps': 700000}, {'reward': -9, 'timesteps': 300000}, {'reward': -8, 'timesteps': 6600000}, {'reward': -7, 'timesteps': 500000}, {'reward': -6, 'timesteps': 100000}, {'reward': -5, 'timesteps': 200000}]
Next reward target: -4


In [ ]:
#del model # remove to demonstrate saving and loading

#model = A2C.load("a2c_pong")

obs = env.reset()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
env.reset()

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to target thread (0x55581a9bdd70)

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to target thread (0x55581a9bdd70)

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to target thread (0x55581a9bdd70)

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to target thread (0x55581a9bdd70)

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to target thread (0x55581a9bdd70)

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to target thread (0x55581a9bdd70)

QObject::moveToThread: Current thread (0x55581a9bdd70) is not the object's thread (0x55581bfa6030).
Cannot move to tar